In [107]:
%run PVModel_functions.ipynb

In [108]:
%run PVsystem_filterfunction_jack-may.ipynb

In [109]:
#This cell imports all the necessary packages & module data
import pandas as pd 
import numpy as np
from datetime import datetime
import datatools
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import pvlib
import pvanalytics
import math
import pvpltools
import seaborn as sns
import sklearn
from sklearn.metrics import r2_score
from pvpltools import iec61853
from pvpltools.module_efficiency import adr, heydenreich, motherpv, pvgis, mpm5, mpm6, bilinear
from pvpltools.module_efficiency import fit_efficiency_model, fit_bilinear

%matplotlib inline 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

mpl.rcParams['font.size']=12
mpl.rcParams['lines.linewidth']=1
mpl.rcParams['xtick.labelsize']=10
#mpl.rcParams['font.weight']='bold'
mpl.rcParams['axes.titlesize']=22

In [321]:
met_name='dbo.LVRM_MET_DataOut'
met2_name = 'dbo.LVRM_FastTrackMet_FastTrackMet_1m'

In [322]:
start = '2018-01-01 00:00:00-07:00'
end = '2019-01-01 00:00:00-07:00'

In [323]:
met = sys_met_data(mettablename=met_name, systablename=met2_name, start=start, end=end)

In [324]:
#Import 1-min System Data
tablename = 'dbo.LVRM_Baseline6kW_Baseline'
sys_sql = f"select * from {tablename} where TmStamp between '{start}' and '{end}';"
#met_sql = f"select * from {tablename}"
sys = pd.read_sql(sys_sql, engine, index_col='TmStamp')
sys.index = pd.DatetimeIndex(sys.index)
sys.index = sys.index.tz_localize('MST') # pvlib requires indexes to be timezone aware or it assumes UTC

In [325]:
df = sys.merge(met, how='inner', left_index=True, right_index=True)

In [326]:
df

,RecNum,Sys1Vdc_Avg,Sys1Str1Idc_Avg,Sys1Str2Idc_Avg,Sys1Vac_Avg,Sys1Iac_Avg,Sys1Wac_Avg,Sys1VARac_Avg,Sys1PowerFactor_Avg,Sys1Frequency_Avg,...,Temp_BPI_C_Avg,RecNum_y,AmbientTemp_Avg,BroadbandPOA_Wm2_Avg,FrontRC_R1B18_Clean_E_Avg,FrontRC_R1B18_Clean_Temp_Avg,RearRC_R1B18_Hi_E_Avg,RearRC_R1B18_Hi_Temp_Avg,RearRC_R1B18_Lo_E_Avg,RearRC_R1B18_Lo_Temp_Avg
TmStamp,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00-07:00,1097313.0,4.6,0.00900,0.006,244.907349,0.517306,-2.859345,-126.591919,-0.022000,59.973515,...,9.980,208244.0,9.316667,-2.005461,0.0,NaN,0.0,NaN,0.0,NaN
2018-01-01 00:01:00-07:00,1097314.0,4.6,0.00900,0.006,244.974854,0.517257,-2.859638,-126.659050,-0.022591,59.978344,...,9.970,208245.0,9.121667,-2.050670,0.0,NaN,0.0,NaN,0.0,NaN
2018-01-01 00:02:00-07:00,1097315.0,4.6,0.00875,0.006,244.998596,0.517478,-2.933352,-126.674141,-0.022548,59.986851,...,9.910,208246.0,9.118334,-2.085029,0.0,NaN,0.0,NaN,0.0,NaN
2018-01-01 00:03:00-07:00,1097316.0,4.6,0.00900,0.006,244.813232,0.517131,-3.006441,-126.605370,-0.022642,59.982136,...,9.830,208247.0,8.916667,-2.179063,0.0,NaN,0.0,NaN,0.0,NaN
2018-01-01 00:04:00-07:00,1097317.0,4.6,0.00900,0.006,244.253403,0.516095,-2.860059,-125.907745,-0.022688,59.987041,...,9.770,208248.0,9.033334,-2.226080,0.0,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 23:56:00-07:00,428214.0,1.0,0.00600,0.003,245.174988,0.505246,-2.933434,-123.759796,-0.023685,59.988529,...,2.900,373863.0,2.746667,-2.193530,0.0,NaN,0.0,NaN,0.0,NaN
2018-12-31 23:57:00-07:00,428215.0,1.0,0.00600,0.003,245.183121,0.505393,-2.860179,-123.711060,-0.024266,59.989731,...,2.943,373864.0,2.800000,-2.130238,0.0,NaN,0.0,NaN,0.0,NaN
2018-12-31 23:58:00-07:00,428216.0,1.0,0.00600,0.003,245.250107,0.505684,-2.860193,-123.932838,-0.024244,59.990246,...,3.008,373865.0,2.790000,-2.070562,0.0,NaN,0.0,NaN,0.0,NaN


In [327]:
df.to_hdf('RTC_nevada.h5', key='df', complib='zlib', complevel=5)

In [258]:
df = df[~df.index.duplicated(keep='first')]

In [251]:
df.columns

Index(['RecNum', 'Sys1Vdc_Avg', 'Sys1Str1Idc_Avg', 'Sys1Str2Idc_Avg',
       'Sys1Vac_Avg', 'Sys1Iac_Avg', 'Sys1Wac_Avg', 'Sys1VARac_Avg',
       'Sys1PowerFactor_Avg', 'Sys1Frequency_Avg', 'ModTemp1_Avg',
       'ModTemp2_Avg', 'ModTemp3_Avg', 'ModTemp4_Avg', 'ModTemp5_Avg',
       'ModTemp6_Avg', 'ModTemp7_Avg', 'ModTemp8_Avg', 'Sys2Vdc_Avg',
       'Sys2Str1Idc_Avg', 'Sys2Str2Idc_Avg', 'Sys2Vac_Avg', 'Sys2Iac_Avg',
       'Sys2Wac_Avg', 'Sys2VARac_Avg', 'Sys2PowerFactor_Avg',
       'Sys2Frequency_Avg', 'ModTemp9_Avg', 'ModTemp10_Avg', 'ModTemp11_Avg',
       'ModTemp12_Avg', 'ModTemp13_Avg', 'ModTemp14_Avg', 'ModTemp15_Avg',
       'ModTemp16_Avg', 'RefCell1Irrad_Avg', 'RefCell1Temp_Avg',
       'RefCell1R_Avg', 'RefCell2Irrad_Avg', 'RefCell2Temp_Avg',
       'RefCell2R_Avg', 'LocalAmbientTemp_Avg', 'POAIrrad1_Avg',
       'ICP7019_1CJCTemp_Avg', 'ICP7019_2CJCTemp_Avg', 'ICP7019_3CJCTemp_Avg',
       'ICP7019_4CJCTemp_Avg', 'ICP7019_5CJCTemp_Avg', 'ICP7019_6CJCTemp_Avg',
       'IC